In [1]:
import os

In [2]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\AI Personal Projects\\Machine Learning Projects\\Customer Churn Prediction\\Customer-Churn-Prediction-'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from CustomerChurn import logger
from CustomerChurn.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    def download_file(self):
        # os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded with the following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-18 15:16:28,363] 32 CustomerChurnLogger - common - INFO - yaml file: config\config.yaml loaded successfully
[2025-07-18 15:16:28,365] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2025-07-18 15:16:28,367] 32 CustomerChurnLogger - common - INFO - yaml file: schema.yaml loaded successfully
[2025-07-18 15:16:28,369] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts
[2025-07-18 15:16:28,371] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts/data_ingestion
[2025-07-18 15:16:29,345] 14 CustomerChurnLogger - 2518646032 - INFO - artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 159758
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "da62fa5c84c7a06b02c66385022dee04747667129853d3195eba35badd9332c0"
Strict-Transport-Security: max-age=31536000
X-Content-Typ